# Auto-analyze Zoom chats

Find who was most active on Zoom

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import re

In [ ]:
folder_name = '../../data/attendance/chats'

In [ ]:
# Create a list of folders to analyze

token = 'Neuroscience'

file_list = os.listdir(folder_name)
file_list = [s for s in file_list if token in s]
# file_list

In [ ]:
# Populate the dataframe

df = pd.DataFrame(columns=['date','name'])

for dirname in file_list:
    fname = folder_name + '/' + dirname + '/meeting_saved_chat.txt'
    date = dirname.split(' ')[0]
    with open(fname, 'r', encoding='Utf8') as f:
        already_included = set()
        for l in f: # Every row
            #print(l.split(' : ')[0].split('From ')[1])            
            name = re.findall(r"From .+ : ", l)
            if name:               
                name = name[0][5:-3].strip()
                if 'Privately' in name:
                    name = name.split(' to ')[0].strip()            
                if name not in already_included:
                    #print(name)
                    already_included |= {name}
                    df = df.append({'date':date, 'name':name}, ignore_index=True)

In [ ]:
print('Total different names:', len(df.name.unique()))
print(f"Total different meetings: {len(df.date.unique())}")

In [ ]:
# sorted(df.name.unique())

In [ ]:
# df.query("name=='Arseny Khakhalin'")

In [ ]:
df2 = (df
       .groupby(['date','name']) # Some days have more than one log
       .agg('first')
       .reset_index()
       .groupby('name')
       .agg({'date': 'count'})
       .reset_index()
       .rename({'date':'number'}, axis=1)
       .sort_values(by='number', ascending=False)
)

df3 = (df2
       .groupby('number')
       .agg({'name': lambda x: ', '.join(x)}) 
       .sort_values('number', ascending=False)
       .reset_index()
      )

for i in range(df3.shape[0]):
    print(df3.loc[i, 'number'], ':', df3.loc[i, 'name'])

In [ ]:
# Save data
df2.to_csv('../../data/attendance/zoom_chats.csv', index=False)